<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
from qlib.contrib.model.gbdt import LGBModel
from qlib.contrib.data.handler import Alpha158
from qlib.utils import init_instance_by_config, flatten_dict
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord, SigAnaRecord
import qlib
import pandas as pd
from qlib.contrib.strategy import TopkDropoutStrategy
from qlib.contrib.evaluate import (
    backtest_daily as normal_backtest,
    risk_analysis,
)
from qlib.contrib.report import analysis_model, analysis_position
import importlib.util
#import quantstats as qs
from pandas_profiling import ProfileReport
#from utils.fix.fix_pandas_profiling import *
from qlib.data.dataset import DataHandlerLP

%matplotlib inline

Please install necessary libs for CatBoostModel.
ModuleNotFoundError. XGBModel is skipped(optional: maybe installing xgboost can fix it).


/home/SENSETIME/liudavide/anaconda3/envs/backtesting/lib/python3.8/site-packages/plotly-5.9.0-py3.8.egg/plotly/graph_objs/__init__.py:288: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

/home/SENSETIME/liudavide/anaconda3/envs/backtesting/lib/python3.8/site-packages/seaborn/rcmod.py:82: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

/home/SENSETIME/liudavide/anaconda3/envs/backtesting/lib/python3.8/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [10]:
from qlib.data.dataset.loader import QlibDataLoader
from qlib.data.dataset.processor import ZScoreNorm, Fillna, CSZScoreNorm, DropnaLabel

market = "csi300"  # or csi500
benchmark = "SH000300"  # must be one of the codes included in the market
train = ["2013-01-01", "2015-01-29"]
valid = ["2015-01-31", "2016-12-31"]
test = ["2017-01-01", "2020-08-01"]

data_handler_config = {
    "start_time": train[0],
    "end_time": test[1],
    #"fit_start_time": train[0],
    #"fit_end_time": train[1],
    "instruments": market,
    "data_loader": {
        "class": QlibDataLoader,
        "kwargs": {
            "config": {
                # all alphas operators: https://github.com/microsoft/qlib/blob/main/qlib/data/ops.py
                "label": "Ref($close, -2)/Ref($close, -1) - 1",
                #"feature": (["Ref($close, -2)/Ref($close, -1) - 1"], ['aa']),
                    #("LABEL0"),
                #],
            "freq": "day",
            },
        },
    },
    'learn_processors': [
        #{"class": "ProcessInf", "kwargs": {}},
        #{"class": "ZScoreNorm", "kwargs": {}},
        #{"class": "Fillna", "kwargs": {}},
        {"class": "DropnaLabel", "kwargs": {}},
    ]
}
task = {
    "model": {
        "class": "LGBModel",
        "module_path": "qlib.contrib.model.gbdt",
        "kwargs": {
            "loss": "mse",
            "colsample_bytree": 0.8879,
            "learning_rate": 0.0421,
            "subsample": 0.8789,
            "lambda_l1": 205.6999,
            "lambda_l2": 580.9768,
            "max_depth": 8,
            "num_leaves": 210,
            "num_threads": 20,
        },
    },
    "dataset": {
        "class": "DatasetH",
        "module_path": "qlib.data.dataset",
        "kwargs": {
            "handler": {
                #"class": "Alpha360",
                #"class": "Alpha158",
                'class': DataHandlerLP,
                #"module_path": "qlib.contrib.data.handler",
                "module_path": qlib.data.dataset.handler,
                "kwargs": data_handler_config,
            },
            "segments": {
                "train": tuple(train),
                "valid": tuple(valid),
                "test": tuple(test),
            },
        },
    },
}
port_analysis_config = {
    "executor": {
        "class": "SimulatorExecutor",
        "module_path": "qlib.backtest.executor",
        "kwargs": {
            "time_per_step": "day",
            "generate_portfolio_metrics": True,
        },
    },
    "strategy": {
        "class": "TopkDropoutStrategy",
        "module_path": "qlib.contrib.strategy.signal_strategy",
        "kwargs": {
            "signal": "<PRED>",
            "topk": 50,
            "n_drop": 5,
        },
    },
    "backtest": {
        "start_time": test[0],
        "end_time": test[1],
        "account": 100000000,
        "benchmark": benchmark,
        "exchange_kwargs": {
            "freq": "day",
            "limit_threshold": 0.095,
            "deal_price": "close",
            "open_cost": 0.0005,
            "close_cost": 0.0015,
            "min_cost": 5,
        },
    },
}


In [3]:
# retrive the data
# run this command only once
#!python qlib-main/scripts/get_data.py qlib_data --target_dir ~/.qlib/qlib_data/cn_data --region cn

In [4]:
qlib.init()

[6475:MainThread](2022-09-01 16:51:02,799) INFO - qlib.Initialization - [config.py:413] - default_conf: client.
[6475:MainThread](2022-09-01 16:51:02,805) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[6475:MainThread](2022-09-01 16:51:02,807) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/home/SENSETIME/liudavide/.qlib/qlib_data/cn_data')}


In [ ]:
# example: config_lightgbm_13_alphas.py
c = input("Input path to config: ")

In [ ]:
# load config
config_path = str(c)
config_name = str(config_path.split('/')[-1].split('.')[0])
spec = importlib.util.spec_from_file_location(config_name, config_path)
config = importlib.util.module_from_spec(spec)
spec.loader.exec_module(config)

In [11]:
ds = init_instance_by_config(task["dataset"])

TypeError: Unsupported type

In [ ]:
# in [train, valid, test]
segment = input("Input dataset segment: ")

In [ ]:
df = ds.prepare(segment)
df

In [ ]:
df.describe()

In [ ]:
# example: "SH000300", "SH600000"
stock = input("Input stock name: ")

In [ ]:
df_stock = df[df.index.get_level_values('instrument') == stock]
#profile = ProfileReport(df_stock, title="Pandas Profiling Report")
#profile.to_notebook_iframe()
df_stock